In [8]:
import polars as pl
import numpy as np
import dcor
import seaborn as sns
import matplotlib.pyplot as plt

from scipy.stats import norm
from src.util.constants import DATA_PATH

### Load data

In [9]:
df_train = pl.read_parquet(DATA_PATH / 'df_train.parquet')

In [3]:
df_train.head()

date_id,time_id,symbol_id,weight,feature_00,feature_01,feature_02,feature_03,feature_04,feature_05,feature_06,feature_07,feature_08,feature_09,feature_10,feature_11,feature_12,feature_13,feature_14,feature_15,feature_16,feature_17,feature_18,feature_19,feature_20,feature_21,feature_22,feature_23,feature_24,feature_25,feature_26,feature_27,feature_28,feature_29,feature_30,feature_31,feature_32,…,feature_51,feature_52,feature_53,feature_54,feature_55,feature_56,feature_57,feature_58,feature_59,feature_60,feature_61,feature_62,feature_63,feature_64,feature_65,feature_66,feature_67,feature_68,feature_69,feature_70,feature_71,feature_72,feature_73,feature_74,feature_75,feature_76,feature_77,feature_78,responder_0,responder_1,responder_2,responder_3,responder_4,responder_5,responder_6,responder_7,responder_8
i16,i16,i8,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,i8,i8,i16,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,…,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32
0,0,1,3.889038,null,null,null,null,null,0.851033,0.242971,0.2634,-0.891687,11,7,76,-0.883028,0.003067,-0.744703,null,-0.169586,null,-1.335938,-1.707803,0.91013,null,1.636431,1.522133,-1.551398,-0.229627,null,null,1.378301,-0.283712,0.123196,null,null,…,0.204797,null,null,-0.808103,null,-2.037683,0.727661,null,-0.989118,-0.345213,-1.36224,null,null,null,null,null,-1.251104,-0.110252,-0.491157,-1.02269,0.152241,-0.659864,null,null,-0.261412,-0.211486,-0.335556,-0.281498,0.738489,-0.069556,1.380875,2.005353,0.186018,1.218368,0.775981,0.346999,0.095504
0,0,7,1.370613,null,null,null,null,null,0.676961,0.151984,0.192465,-0.521729,11,7,76,-0.865307,-0.225629,-0.582163,null,0.317467,null,-1.250016,-1.682929,1.412757,null,0.520378,0.744132,-0.788658,0.641776,null,null,0.2272,0.580907,1.128879,null,null,…,1.172836,null,null,-1.625862,null,-1.410017,1.063013,null,0.888355,0.467994,-1.36224,null,null,null,null,null,-1.065759,0.013322,-0.592855,-1.052685,-0.393726,-0.741603,null,null,-0.281207,-0.182894,-0.245565,-0.302441,2.965889,1.190077,-0.523998,3.849921,2.626981,5.0,0.703665,0.216683,0.778639
0,0,9,2.285698,null,null,null,null,null,1.056285,0.187227,0.249901,-0.77305,11,7,76,-0.675719,-0.199404,-0.586798,null,-0.814909,null,-1.296782,-2.040234,0.639589,null,1.597359,0.657514,-1.350148,0.364215,null,null,-0.017751,-0.317361,-0.122379,null,null,…,0.535897,null,null,-0.72542,null,-2.29417,1.764551,null,-0.120789,-0.063458,-1.36224,null,null,null,null,null,-0.882604,-0.072482,-0.617934,-0.86323,-0.241892,-0.709919,null,null,0.377131,0.300724,-0.106842,-0.096792,-0.864488,-0.280303,-0.326697,0.375781,1.271291,0.099793,2.109352,0.670881,0.772828
0,0,10,0.690606,null,null,null,null,null,1.139366,0.273328,0.306549,-1.262223,42,5,150,-0.694008,3.004091,0.114809,null,-0.251882,null,-1.902009,-0.979447,0.241165,null,-0.392359,-0.224699,-2.129397,-0.855287,null,null,0.404142,-0.578156,0.105702,null,null,…,2.413415,null,null,1.313203,null,-0.810125,2.939022,null,3.988801,1.834661,-1.36224,null,null,null,null,null,-0.697595,1.074309,-0.206929,-0.530602,4.765215,0.571554,null,null,-0.226891,-0.251412,-0.215522,-0.296244,0.408499,0.223992,2.294888,1.097444,1.225872,1.225376,1.114137,0.775199,-1.379516
0,0,14,0.44057,null,null,null,null,null,0.9552,0.262404,0.344457,-0.613813,44,3,16,-0.947351,-0.030018,-0.502379,null,0.646086,null,-1.844685,-1.58656,-0.182024,null,-0.969949,-0.673813,-1.282132,-1.399894,null,null,0.043815,-0.320225,-0.031713,null,null,…,1.253902,null,null,0.476195,null,-0.771732,2.843421,null,1.379815,0.411827,-1.36224,null,null,null,null,null,-0.948601,-0.136814,-0.447704,-1.141761,0.099631,-0.661928,null,null,3.678076,2.793581,2.61825,3.418133,-0.373387,-0.502764,-0.348021,-3.928148,-1.591366,-5.0,-3.57282,-1.089123,-5.0


### Distributions

In [4]:
df_train.describe()

statistic,date_id,time_id,symbol_id,weight,feature_00,feature_01,feature_02,feature_03,feature_04,feature_05,feature_06,feature_07,feature_08,feature_09,feature_10,feature_11,feature_12,feature_13,feature_14,feature_15,feature_16,feature_17,feature_18,feature_19,feature_20,feature_21,feature_22,feature_23,feature_24,feature_25,feature_26,feature_27,feature_28,feature_29,feature_30,feature_31,…,feature_51,feature_52,feature_53,feature_54,feature_55,feature_56,feature_57,feature_58,feature_59,feature_60,feature_61,feature_62,feature_63,feature_64,feature_65,feature_66,feature_67,feature_68,feature_69,feature_70,feature_71,feature_72,feature_73,feature_74,feature_75,feature_76,feature_77,feature_78,responder_0,responder_1,responder_2,responder_3,responder_4,responder_5,responder_6,responder_7,responder_8
str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""count""",3.2984858e7,3.2984858e7,3.2984858e7,3.2984858e7,2.9802806e7,2.9802806e7,2.9802806e7,2.9802806e7,2.9802806e7,3.2984858e7,3.2984858e7,3.2984858e7,3.2888859e7,3.2984858e7,3.2984858e7,3.2984858e7,3.2984858e7,3.2984858e7,3.2984858e7,3.2126199e7,3.2984597e7,3.284146e7,3.2984632e7,3.2984632e7,3.2984858e7,2.4628249e7,3.2984858e7,3.2984858e7,3.2984858e7,3.2984858e7,2.4628249e7,2.4628249e7,3.2984858e7,3.2984858e7,3.2984858e7,2.4628249e7,…,3.2971053e7,3.220294e7,2.972424e7,3.2971053e7,3.220294e7,3.2984632e7,3.2984632e7,3.2645614e7,3.2984858e7,3.2984858e7,3.2984858e7,3.2692164e7,3.2757388e7,3.2747296e7,3.2668029e7,3.2668029e7,3.2984858e7,3.2984858e7,3.2984858e7,3.2984858e7,3.2984858e7,3.2984858e7,3.2642355e7,3.2642355e7,3.295625e7,3.295625e7,3.2971421e7,3.2971421e7,3.2984858e7,3.2984858e7,3.2984858e7,3.2984858e7,3.2984858e7,3.2984858e7,3.2984858e7,3.2984858e7,3.2984858e7
"""null_count""",0.0,0.0,0.0,0.0,3.182052e6,3.182052e6,3.182052e6,3.182052e6,3.182052e6,0.0,0.0,0.0,95999.0,0.0,0.0,0.0,0.0,0.0,0.0,858659.0,261.0,143398.0,226.0,226.0,0.0,8.356609e6,0.0,0.0,0.0,0.0,8.356609e6,8.356609e6,0.0,0.0,0.0,8.356609e6,…,13805.0,781918.0,3.260618e6,13805.0,781918.0,226.0,226.0,339244.0,0.0,0.0,0.0,292694.0,227470.0,237562.0,316829.0,316829.0,0.0,0.0,0.0,0.0,0.0,0.0,342503.0,342503.0,28608.0,28608.0,13437.0,13437.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"""mean""",790.553554,462.362576,17.718772,1.828102,0.122263,-0.01368,0.124301,0.123898,-0.004374,-0.028146,-0.004135,-0.010677,0.057588,32.881209,4.840214,172.225653,-0.022161,-0.015885,-0.024635,-0.068272,-0.052487,-0.064351,0.012252,-0.005214,-0.169221,0.019181,0.030641,0.053206,0.137534,0.027036,-0.045934,-0.149635,-0.066765,-0.066517,-0.06216,0.016709,…,-0.026979,-0.038404,0.034889,0.019502,0.031358,-0.017548,0.010742,-0.009973,-0.001921,-0.004928,-0.000539,-0.095212,-0.074635,-0.084126,-0.009926,-0.014039,-0.021379,-0.013176,-0.022215,-0.020492,-0.013323,-0.023196,-0.036002,-0.038443,-0.023147,-0.025374,-0.026904,-0.029127,-0.001292,-0.001177,-0.000206,-0.004332,-0.004164,-0.003101,-0.002612,0.000097,-0.001353
"""std""",351.980587,269.248996,11.305788,1.037553,1.036687,1.048828,1.036778,1.036302,1.024007,1.013443,1.033632,1.030019,1.006277,24.105205,2.964258,165.455039,0.966445,0.97566,0.973699,1.014811,1.004281,1.012696,1.035698,1.0528,0.974685,1.080171,0.993629,0.98714,1.212492,1.013893,1.025038,1.068237,1.031001,0.988799,1.034836,0.884922,…,1.010242,1.028723,1.037942,1.010217,1.027041,1.015008,1.034891,0.975516,0.973799,0.975934,1.106911,0.84264,0.846316,0.845181,1.041653,1.052071,0.967034,0.969113,0.971065,0.964999,0.964864,0.971879,0.92723,0.924615,0.925642,0.922084,0.920019,0.920847,0.628466,0.6279,0.635972,0.77962,0.870673,0.672516,0.909948,0.937307,0.885248
"""min""",0.0,0.0,0.0,0.149967,-5.794129,-5.741592,-5.72601,-5.60189,-5.79988,-25.350399,-58.604362,-39.085358,-15.566865,2.0,1.0,9.

Most features exhibit strong outliers (100x the standard deviation), i.e. their distributions have fat tails.

In [ ]:
# compare the observed distribution of responder_6 with a normal distribution
sns.histplot(data=df_train, x='responder_6', stat='density').set(title='Distribution of responder_6')

# Calculate mean and standard deviation
mean = df_train['responder_6'].mean()
std = df_train['responder_6'].std()

# Create a range for x-axis
xmin, xmax = plt.xlim()
x = np.linspace(xmin, xmax, 100)

# Compute the normal distribution PDF
p = norm.pdf(x, mean, std)

# Overlay the normal distribution curve
plt.plot(x, p, 'k', linewidth=2)

# Display the plot
plt.show()

Responder_6 has higher kurtosis than a normal distribution.

### Correlation of features with responder_6

In [ ]:
results = []

for i in range(79):
    # get i as a two-digit string with leading zeros if necessary
    responder_name = f'feature_{i:02}'
    print(f'{responder_name}')
    
    df_subset = df_train.select(['responder_6', responder_name]).filter(pl.col(responder_name).is_not_null())
    
    if df_subset[responder_name].dtype == pl.Int8 or df_subset[responder_name].dtype == pl.Int16:
        df_subset = df_subset.with_columns(df_subset[responder_name].cast(pl.Float32))
        print(f'{responder_name} was casted to float32')
    
    pearson_corr = np.corrcoef(df_subset[responder_name].to_numpy(), df_subset['responder_6'].to_numpy())[0, 1]
    dist_corr = dcor.distance_correlation(np.array(df_subset[responder_name], dtype='float64'), np.array(df_subset['responder_6'], dtype='float64'))
    
    results.append([responder_name, pearson_corr, dist_corr])
    
df_results = pl.DataFrame(results, ['feature', 'pearson_correlation', 'distance_correlation'], orient='row')

In [ ]:
df_results.sort('pearson_correlation').tail()

In [ ]:
df_results.sort('distance_correlation').tail()

In [ ]:
df_results.describe()

The maximum distance correlation is .146, the maximum Pearson correlation is .022.

According to distance correlation, the 5 most promising features are:
- 14
- 69
- 72
- 13
- 68

According to distance correlation, the 5 most promising features are:
- 60
- 51
- 59
- 58
- 54

All of these features are almost available.

### Correlation of responders with responder_6

In [ ]:
results = []

for i in range(9):
    if i == 6:
        continue
    
    responder_name = f'responder_{i}'
    print(f'{responder_name}')

    df_subset = df_train.select(['responder_6', responder_name]).filter(pl.col(responder_name).is_not_null())

    if df_subset[responder_name].dtype == pl.Int8 or df_subset[responder_name].dtype == pl.Int16:
        df_subset = df_subset.with_column(df_subset[responder_name].cast(pl.Float32))
        print(f'{responder_name} was casted to float32')

    pearson_corr = np.corrcoef(df_subset[responder_name].to_numpy(), df_subset['responder_6'].to_numpy())[0, 1]
    dist_corr = dcor.distance_correlation(np.array(df_subset[responder_name], dtype='float64'),
                                          np.array(df_subset['responder_6'], dtype='float64'))

    results.append([responder_name, pearson_corr, dist_corr])

df_results = pl.DataFrame(results, ['responder', 'pearson_correlation', 'distance_correlation'], orient='row')

In [ ]:
df_results

There is strong correlation with other responders. Distance correlation is between .09 and .81. However, these are not available at prediction time. We'll also look into correlation with the lags below.

### Stationarity of responder_6

In [ ]:
df_responder_by_date = df_train.group_by('date_id').agg(pl.col('responder_6').mean().alias('responder_6_mean'), pl.col('responder_6').std().alias('responder_6_std'))

In [ ]:
df_responder_by_date.plot.line(x='date_id', y='responder_6_mean')

In [ ]:
df_responder_by_date.plot.line(x='date_id', y='responder_6_std')

Despite an unstable collection of symbols, the mean and standard deviation of the responder look relatively stable.

### Stationarity of features

In [ ]:
feature_number = 0
feature = f'feature_{feature_number:02}' 

df_feature_by_date = df_train.group_by('date_id').agg(pl.col(feature).mean().alias('feature_mean'), pl.col(feature).std().alias('feature_std'))

df_feature_by_date.plot.line(x='date_id', y='feature_mean').show()
df_feature_by_date.plot.line(x='date_id', y='feature_std').show()

Features 0, 2, 3, 15, 16, 17, 20, 22, 23, 24, 25, 27, 28, 29, 30 show some variation over time, but no clear pattern.

Features 26, 44, 62, 63, 64 show a decreasing mean. This may be due to the addition of new symbols over time.

Features 32, 34, 35, 61 may show a seasonal pattern.

Features 39, 40, 41, 42, 43, 44, 45, 62, 63, 64 show a decreasing std. (Other variables also seem to have a decreasing std, but not as pronounced.)  This may be due to the addition of new symbols over time.

### Responder by symbol

In [ ]:
df_responder_by_symbol = df_train.group_by('symbol_id').agg(pl.col('responder_6').mean().alias('responder_6_mean'), pl.col('responder_6').std().alias('responder_6_std'))
df_responder_by_symbol.describe()

The mean is (basically) zero for all symbols. The standard deviation varies between 0.7 and 1.1, also no huge variation.

We'll use the `symbol_id` as a categorical variable in the model.

### Correlation between symbols

In [ ]:
df_symbols = df_train.select('date_id', 'time_id', 'symbol_id', 'responder_6')

In [ ]:
# convert from long format to wide format
df_symbols_pivot = df_symbols.pivot('symbol_id', index=['date_id', 'time_id'], values='responder_6').select(pl.exclude(['date_id', 'time_id']))

In [ ]:
# re-sort columns
columns = [str(i) for i in range(39)]
df_symbols_pivot = df_symbols_pivot.select(columns)

In [ ]:
# we'll cheat here a bit for simplicity and only consider rows for which we have all symbols
df_corr = df_symbols_pivot.drop_nulls().corr()

In [ ]:
# heatmap of correlation matrix - only the lower triangle without the diagonal
mask = np.triu(np.ones_like(df_corr.to_numpy()), k=0)
plt.figure(figsize=(20, 12))

# show the values in the heatmap, rounded to two decimals
sns.heatmap(df_corr.to_numpy(), annot=True, fmt=".2f", mask=mask, cmap='coolwarm')
plt.show()



A few symbols are highly correlated, but most responders show very low correlation.

### Autocorelation and correlation with lags of other symbols

In [ ]:
# Get the lag of responder_6 by one date_id, grouped by time_id and symbol_id
df_train_with_lag = df_train.with_columns(pl.col('responder_6').shift(1).over(['time_id', 'symbol_id']).alias('responder_6_lag'))

In [ ]:
df_train_with_lag.select(pl.corr("responder_6", "responder_6_lag"))

In [ ]:
df_train_with_lag_subset = df_train_with_lag.select(["responder_6", "responder_6_lag"]).drop_nulls()

dcor.distance_correlation(np.array(df_train_with_lag_subset['responder_6'], dtype='float64'), np.array(df_train_with_lag_subset['responder_6_lag'], dtype='float64'))

The correlation between responder_6 and its 'one-day' lag is very low, although there is some distance correlation. We'll use it as a feature.

In [ ]:
# Get the lag of responder_6 by one time_id, grouped by symbol_id
df_train_with_time_lag = df_train.with_columns(pl.col('responder_6').shift(1).over(['symbol_id']).alias('responder_6_lag'))

In [ ]:
df_train_with_time_lag.select(pl.corr("responder_6", "responder_6_lag"))

The correlation with the 'one-time' lag is very high, but will usually not be available.

In [ ]:
df_train_with_time_lag.filter(pl.col('time_id') == 0).select(pl.corr("responder_6", "responder_6_lag"))

The first time in the morning does not seem to be correlated with the last time the 'day' before.

In [ ]:
df_train_with_mean = df_train.with_columns(pl.col('responder_6').mean().over(['date_id', 'symbol_id']).alias('responder_6_mean'))
df_train_with_mean.select("responder_6", "responder_6_mean").describe()

The day-average of the responder is usually close to zero, but can vary between -3.75 and +3.5.

In [ ]:
df_train_with_mean_lag = df_train_with_mean.with_columns(pl.col('responder_6_mean').shift(1).over(['time_id', 'symbol_id']).alias('responder_6_mean_lag'))

In [ ]:
df_train_with_mean_lag.select(pl.corr("responder_6", "responder_6_mean_lag"))

In [ ]:
df_train_with_mean_lag_subset = df_train_with_mean_lag.select(["responder_6", "responder_6_mean_lag"]).drop_nulls()

dcor.distance_correlation(np.array(df_train_with_mean_lag_subset['responder_6'], dtype='float64'), np.array(df_train_with_mean_lag_subset['responder_6_mean_lag'], dtype='float64'))

There seems to be weak distance correlation with the mean of the previous 'day'. We'll use it as a feature.

In [ ]:
df_train_with_std = df_train.with_columns(pl.col('responder_6').std().over(['date_id', 'symbol_id']).alias('responder_6_std'))
df_train_with_std_lag = df_train_with_std.with_columns(pl.col('responder_6_std').shift(1).over(['time_id', 'symbol_id']).alias('responder_6_std_lag'))

In [ ]:
df_train_with_std_lag.select(pl.corr("responder_6", "responder_6_std_lag"))

In [ ]:
df_train_with_std_lag_subset = df_train_with_std_lag.select(["responder_6", "responder_6_std_lag"]).drop_nulls()

dcor.distance_correlation(np.array(df_train_with_std_lag_subset['responder_6'], dtype='float64'), np.array(df_train_with_std_lag_subset['responder_6_std_lag'], dtype='float64'))

There seems to be quite some distance correlation with the std of the previous 'day'. We'll use it as a feature.

In [ ]:
df_train_responders = df_train.select('date_id', 'time_id', 'symbol_id', 'responder_6')

In [ ]:
for i in range(39):
    df_train_responders = df_train_responders.with_columns(pl.col('responder_6').filter(pl.col('symbol_id') == i).mean().over(['date_id']).alias(f'responder_6_symbol_{i}_mean'))
    df_train_responders = df_train_responders.with_columns(pl.col(f'responder_6_symbol_{i}_mean').shift(1).over(['time_id', 'symbol_id']).alias(f'responder_6_symbol_{i}_mean_lag')).drop(pl.col(f'responder_6_symbol_{i}_mean'))

In [ ]:
# get the distance correlation of responder_6 with all responder_6_symbol_i_mean_lags
results = []

for i in range(39):
    print(i)
    df_symbol_subset = df_train_responders.filter(pl.col('symbol_id') == i)
    
    results_subset = []
    for i in range(39):
        df_symbol_lag_subset = df_symbol_subset.select(['responder_6', f'responder_6_symbol_{i}_mean_lag']).drop_nulls()
        dist_corr = dcor.distance_correlation(np.array(df_symbol_lag_subset[f'responder_6'], dtype='float64'), np.array(df_symbol_lag_subset[f'responder_6_symbol_{i}_mean_lag'], dtype='float64'))
        
        results_subset.append(dist_corr)
    results.append(results_subset)

df_results = pl.DataFrame(results, [f'responder_6_symbol_{i}_mean_lag' for i in range(39)], orient='row')

In [ ]:
# heatmap of distance correlations
plt.figure(figsize=(20, 12))

# show the values in the heatmap, rounded to two decimals
sns.heatmap(df_results.to_numpy(), annot=True, fmt=".2f", cmap='coolwarm')
plt.show()

Distance correlations are all lower than .08. We'll still use these lags as features and see what happens during feature selection.

(Pearson correlations are all very low--between -.09 and .03)

### Correlation with lags of other responders (by symbol)

In [ ]:
for i in range(9):
    if i == 6:
        continue
        
    df_train_with_mean = df_train.with_columns(pl.col(f'responder_{i}').mean().over(['date_id', 'symbol_id']).alias(f'responder_{i}_mean'))
    df_train_with_mean_lag = df_train_with_mean.with_columns(pl.col(f'responder_{i}_mean').shift(1).over(['time_id', 'symbol_id']).alias(f'responder_{i}_mean_lag'))
    df_train_with_mean_lag_subset = df_train_with_mean_lag.select(["responder_6", f'responder_{i}_mean_lag']).drop_nulls()
    
    distance_corr = dcor.distance_correlation(np.array(df_train_with_mean_lag_subset['responder_6'], dtype='float64'), np.array(df_train_with_mean_lag_subset[f'responder_{i}_mean_lag'], dtype='float64'))
    
    print(f'distance correlation with lagged mean of responder_{i}: {distance_corr}')
    

There seems to be low signal in these lagged means. We'll use them as features and see what happens during feature selection.

### Some feature engineering

In [ ]:
# Mean, max, min, and std of features
feature_columns = [f'feature_{i:02}' for i in range(79) if i not in [9, 10, 11]]  # exclude the integer columns
df_train_with_features = df_train.with_columns(
    feature_mean=pl.mean_horizontal(feature_columns),
    feature_min=pl.min_horizontal(feature_columns),
    feature_max=pl.max_horizontal(feature_columns),
    feature_std=pl.concat_list(feature_columns).list.std()
)

In [ ]:
for feature in ['feature_mean', 'feature_min', 'feature_max', 'feature_std']:
    df_train_with_features_subset = df_train_with_features.select(["responder_6", feature]).drop_nulls()
    
    distance_corr = dcor.distance_correlation(np.array(df_train_with_features_subset['responder_6'], dtype='float64'), np.array(df_train_with_features_subset[feature], dtype='float64'))
    
    print(f'distance correlation with {feature}: {distance_corr}')

These look like great features, let's use them.

### Correlation with fundamental variables

In [ ]:
for feature in ['date_id', 'time_id', 'symbol_id', 'weight']:
    df_train_with_features_subset = df_train_with_features.select(["responder_6", feature]).drop_nulls()
    
    distance_corr = dcor.distance_correlation(np.array(df_train_with_features_subset['responder_6'], dtype='float64'), np.array(df_train_with_features_subset[feature], dtype='float64'))
    
    print(f'distance correlation with {feature}: {distance_corr}')

The fundamental variables show low-ish distance correlation. We'll still use them as features and see what happens during feature selection. Exception: `date_id` since it will not be stationary (and doesn't look too useful).